<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Augmenting-Aerospike-with-Local-Cache¶" data-toc-modified-id="Augmenting-Aerospike-with-Local-Cache¶-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Augmenting Aerospike with Local Cache¶</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#Initialization" data-toc-modified-id="Initialization-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Initialization</a></span><ul class="toc-item"><li><span><a href="#Ensure-database-is-running" data-toc-modified-id="Ensure-database-is-running-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Ensure database is running</a></span></li><li><span><a href="#Connect-to-database." data-toc-modified-id="Connect-to-database.-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Connect to database.</a></span></li><li><span><a href="#Populate-database-with-test-data." data-toc-modified-id="Populate-database-with-test-data.-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Populate database with test data.</a></span></li></ul></li></ul></li><li><span><a href="#Local-Cache" data-toc-modified-id="Local-Cache-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Local Cache</a></span><ul class="toc-item"><li><span><a href="#Simple-Interface" data-toc-modified-id="Simple-Interface-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Simple Interface</a></span><ul class="toc-item"><li><span><a href="#Eviction-Alternatives" data-toc-modified-id="Eviction-Alternatives-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Eviction Alternatives</a></span></li><li><span><a href="#Working-with-Aerospike-TTL-Based-Eviction" data-toc-modified-id="Working-with-Aerospike-TTL-Based-Eviction-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Working with Aerospike TTL Based Eviction</a></span></li></ul></li><li><span><a href="#Comparing-Performance" data-toc-modified-id="Comparing-Performance-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Comparing Performance</a></span><ul class="toc-item"><li><span><a href="#Benefits-of-a-local-cache" data-toc-modified-id="Benefits-of-a-local-cache-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Benefits of a local cache</a></span></li></ul></li></ul></li><li><span><a href="#Cache-Performance" data-toc-modified-id="Cache-Performance-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cache Performance</a></span><ul class="toc-item"><li><span><a href="#The-Model" data-toc-modified-id="The-Model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>The Model</a></span></li><li><span><a href="#Single-Cache" data-toc-modified-id="Single-Cache-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Single Cache</a></span></li><li><span><a href="#Dual-Cache" data-toc-modified-id="Dual-Cache-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Dual Cache</a></span></li></ul></li><li><span><a href="#Takeaways-and-Conclusion" data-toc-modified-id="Takeaways-and-Conclusion-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Takeaways and Conclusion</a></span></li><li><span><a href="#Clean-up" data-toc-modified-id="Clean-up-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Clean up</a></span></li><li><span><a href="#Further-Exploration-and-Resources" data-toc-modified-id="Further-Exploration-and-Resources-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Further Exploration and Resources</a></span><ul class="toc-item"><li><span><a href="#Next-steps" data-toc-modified-id="Next-steps-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Next steps</a></span></li></ul></li></ul></div>

# Augmenting Aerospike with Local Cache¶
This notebook describes the pattern where a local cache is used in front of the Aerospike database. It shows with a simple model how a local cache can enhance performance for various hit ratio and cache speed scenarios.

This notebook requires Aerospike datbase running on localhost and that python and the Aerospike python client have been installed (`pip install aerospike`). Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) for additional details and the docker container.

## Introduction
Caches are ubiquitious. Aerospike is commonly and effectively deployed as a cache in front of a backend database that is remote, slow, and/or limited in throughput. This notebook illustrates use of a local cache that sits in front of the Aerospike database on the client machine. The pattern is applicable for a standalone Aerospike database as well as when Aerospike itself is used as a cache.

A cache provides faster access by keeping the data closer to where it is needed. Examples of caches that are integral part of the computing infrastructure include L1, L2, memory, application specific server caches, and application specific client caches. Cache libraries, external caching servers, and distributed cache infrastructure are deployed for specific caching needs and solutions. Aerospike is designed for fast, reliable, consitent, and cost-effective access across the globally distributed data infrastructure.

This notebook illustrates a simple API for a local cache in front of the Aerospike database.

The notebook is divided in two parts:
1. Part 1 (Section 2) illustrates a simple API for a local cache that front-ends Aerospike database.
1. Part 2 (Section 3) uses a simple model to show the performance impact of a cache in various scenarios.

## Prerequisites
This tutorial assumes familiarity with the following topics:

- Familiarlity with Aerospike and API. See [Hello World](hello_world.ipynb)
- Basic CRUD operations. See [Aerospike Basic Operations](basic_operations.ipynb)

## Initialization

### Ensure database is running
This notebook requires that Aerospike datbase is running. 
[Include the right code cell for Java or Python from the two cells below.] 

In [135]:
!asd >& /dev/null
!pgrep -x asd >/dev/null && echo "Aerospike database is running!" || echo "**Aerospike database is not running!**"

Aerospike database is running!


### Connect to database.
We need a client connected to the database. 

In [136]:
# import the module
from __future__ import print_function
import aerospike

# Configure the client
config = {
  'hosts': [ ('127.0.0.1', 3000) ],
  'policy' : {'key': aerospike.POLICY_KEY_SEND}
}

# Create a client and connect it to the cluster
try:
  client = aerospike.client(config).connect()
except:
  import sys
  print("failed to connect to the cluster with", config['hosts'])
  sys.exit(1)
print('Client successfully connected to the database.')

Client successfully connected to the database.


### Populate database with test data.
The following code populates the test data in set "local-cache-tutorial" in namespace "test". The data consists of a 10000 records with user keys 1-10000 and bins populated with random data.

In [137]:
namespace = 'test'
tutorial_set = 'local_cache_tutorial'
max_data_size = 10000
# Records are addressable via a tuple of (namespace, set, key)

import random
random.seed(1)
try:
    for i in range(max_data_size):
      # Write the records
      client.put((namespace, tutorial_set, 'id-'+str(i+1)), 
                 { 'age': random.randint(20,81),
                   'birth_month': random.choice(['Jan','Feb','Mar','Apr','May','Jun',
                                               'Jul','Aug','Sep','Oct','Nov','Dec']),
                   'gender': random.choice(['M', 'F']),
                   'favorite_colors': random.sample(['red','orange','yellow','green',
                                                   'blue','violet','black','white','grey'], k=3) } )
except Exception as e:
  import sys
  print("error: {0}".format(e), file=sys.stderr)

print('Test data populated.')

Test data populated.


# Local Cache

Here we will illustrate how a local cache can be placed in front of the Aerospike database. Aerospike provides average access time of a millisecond or less for 99%+ requests. A local cache has greater speed advantage - 10-100 times greater - since a cache hit can be in 1-10 microsecond range.

First, let us define the API for the local cache.

## Cache Interface
The cache consists of cache entries with the following key operations:
- get(key)
    - Get from cache if available and TTL not expired, else retrieve from the database and add to the cache.
- update(key, data)
    - Update the database by replacing the current record, and also add or replace the cache entry.
- add(key, entry)
    - If the cache is full, evict appropriate entry and add the cache entry.

### Cache Eviction
A local cache can be implemented with TTL based garbage collection. 

An alternative implementation can maintain a Least-Recently-Used (LRU) list. When the cache is full, the LRU entry is removed to make room for a new entry. LRU can be implemented by maintaining a doubly linked list of cache entries - the classes Cache and CacheEntry allude to the core data structure elements for LRU, but the logic to manage the doubly linked list (which should be straightforward) is not provided here.

A simplistic implementation we have chosen here for illustrative purpose selects a random entry for eviction. A "random" replacement uses dict.popitem(), although better randomized eviction is possible with dict implementations like [randomdict](https://github.com/robtandy/randomdict). 

In [138]:
import time

DEFAULT_TTL = 3600

class CacheEntry:
    _key = None
    _ttl = 0
    _data = None
    # LRU not implemented
    # _lru_prev = None
    # _lru_next = None
    
    def __init__(self, key, ttl, data):
        self._key = key
        self._ttl = ttl
        self._data = data.copy()
        return
        
    
class LocalCache:
    _cache = {}   # dict of key->cache_entry
    _max_size = 0
    # LRU not implemented
    # _lru_head = None
    # _lru_tail = None

    def __init__(self, max_size):
        self._max_size = max_size
        return

    def get(self, key):
        entry = self._cache.get(key)
        if entry is None or entry._ttl < time.time():
            #print("a cache miss", key[2])
            # get it from aerospike database
            _, meta, bins = client.get(key)
            if meta is None:
                #print("entry expired at origin", key[2])
                if entry:
                    self._cache.pop(key)
                return None
            if entry: # in cache, replace
                entry._ttl = time.time()+meta['ttl']
                entry._data = bins
            else: # not in cache, add
                entry = CacheEntry(key, time.time()+meta['ttl'], bins)
                self.add(key, entry)
        #else:
            #print("a cache hit", key[2])
        return entry._data.copy()

    def add(self, key, entry):
        if len(self._cache) == self._max_size:
            #print("cache full, evicting")
            _ = self._cache.popitem() # remove a "random" entry
        self._cache[key] = entry
        #print("added entry", key[2])
        return

    def update(self, key, data):
        # update aerospike database and extend TTL
        meta = {'ttl': DEFAULT_TTL}
        policy = {'exists': aerospike.POLICY_EXISTS_REPLACE}
        try:
            _, meta, _ = client.operate(key, data, meta=meta, policy=policy)
        except:
              print('failed to udpdate database')
              raise
        entry = self._cache.get(key)
        if entry is None:
            entry = CacheEntry(key, time.time()+DEFAULT_TTL, data)
            self.add(key, entry)
        else:
            entry._ttl = time.time()+DEFAULT_TTL
            entry._data = rec.copy()
        return

### Working with Aerospike TTL Based Eviction

Aerospike has TTL based eviction that provides alternatives like: 
1. No eviction or the record is permanent, 
1. At record creation a specific TTL is set, 
1. TTL may be updated any time. 

Since the TTL can change at origin while the record resides in the local cache, the cache will sync up with the origin on TTL at the time of caching. This is no different from when the record is deleted at the origin.

## Comparing Performance
How does a local cache aid performance of Aerospike database? It depends on two key factors. 
1. Cache hit ratio that combines factors like the cache size, invalidation (update) rate, and access pattern. 
1. Latency ratio or the ratio of Aerospike database access latency and local cache access latency. Essentially it is the speedup that a cache provides when there is a cache hit over a cache miss.

Below we run a simple test and compare execution times without cache and with cache. You can experiment with different values of data_size and cache_size to adjust the hit ratio.

In [139]:
data_size = 5000
cache_size = 2000
num_requests = 10000

start_time = time.time()
for i in range(num_requests):
    user_key = 'id-' + str(random.randint(1, data_size))
    key = (namespace, tutorial_set, user_key)
    _ = client.get(key)
time_without_cache = time.time() - start_time
print('Execution time without cache: ', '%5.3fs'%time_without_cache)

start_time = time.time()
cache = LocalCache(cache_size)
for i in range(num_requests):
    user_key = 'id-' + str(random.randint(1, data_size))
    key = (namespace, tutorial_set, user_key)
    _ = cache.get(key)
time_with_cache = time.time() - start_time
print('Execution time with cache: ', '%5.3fs'%time_with_cache)

print('Speedup with cache: ', '%5.1f%%'%((time_without_cache/time_with_cache - 1) * 100))

Execution time without cache:  1.924s
Execution time with cache:  1.515s
Speedup with cache:   27.0%


### Benefits of a local cache

A local cache is beneficial in the following situations, with greater impact as either or both of hit ratio and latency ratio increase.
- If Aerospike is used as a database and not a cache for another database. 
- In cases where Aerospike is used as a cache for another database, the Aerospike cache's hit ratio is high.
- In cases where the hit ratio of either Aerospike or local cache is not high, the local cache advantage is proportionally limited by how large the frequency of access and access time are for the origin database.

# Cache Performance

Aerospike, with its speed at scale advantage, is commonly used as a distributed cache server in front of other databases that are limited in speed, throughput, and cost characteristics. 

We use a simple model to examine how a cache consisting of a local cache and Aerospike database that sits in front of another database.

## The Model
To show the performance gains of a single and dual cache, a simple model is used with two variables for a cache: hit ratio and latency ratio.

Note, both ratios are with respect to the data source (or origin) where the cache goes on a miss. 

## Single Cache
For N random accesses to the origin database, the time needed is N * T, where T is the average access time for the origin database.

With a hit ratio of H and latency ratio of L:
- N\*H requests are directly served from the cache each with T/L access time. The total access time for cache served requests is  N\*H\*T/L.
- The remaining N*(1-H) requests are served from the origin in N\*(1-H)\*T time.
- The total time with a cache is the addition of the above two: N\*H\*T/L + N\*(1-H)\*T.
- The speedup in time and throughput is the ratio of total time without a cache to the total time with a cache. This come to N*T / [N\*H\*T/L + N*\(1-H)\*T] or 1/(1 - H + H/L).

Note, as expected, as H approaches 1 (all requests served from the cache), the speedup approaches L, the latency ratio. Similarly as H approaches 0, the speedup approaches L (no speedup).

This equation applies in both single cache scenarios: either a local cache sitting in front of the Aerospike database, or an Aerospike CacheDB sitting in front of another database.

The following code implements the speedup function with parameters H and L. These are computed below with various H and L values.

In [140]:
import math

def single_cache_speedup(hit_ratio, latency_ratio):
    return 1.0/(1.0 - hit_ratio + hit_ratio / latency_ratio)

hr = [0.1, 0.3, 0.5, 0.7, 0.9]
lr = [10, 100, 1000]
print('LR', 'HR', 'Speedup', sep='\t')
for l in lr:
    for h in hr:
        print(l, h, '%5.0f%%'%((single_cache_speedup(h, l)-1)*100), 
              math.ceil((single_cache_speedup(h, l)-1)*10)*'*', sep="\t")

LR	HR	Speedup
10	0.1	   10%	*
10	0.3	   37%	****
10	0.5	   82%	*********
10	0.7	  170%	******************
10	0.9	  426%	*******************************************
100	0.1	   11%	**
100	0.3	   42%	*****
100	0.5	   98%	**********
100	0.7	  226%	***********************
100	0.9	  817%	**********************************************************************************
1000	0.1	   11%	**
1000	0.3	   43%	*****
1000	0.5	  100%	**********
1000	0.7	  233%	************************
1000	0.9	  891%	******************************************************************************************


Thus, a single cache can provide significant performance and throughput boost in the two scenarios:
- A local cache to the Aerospike database especially if a high hit ratio can be attained.
- Aerospike cache to the backend database because Aerospike provides a large cache size (hence hit ratio) and sub-miilisecond access time.

## Dual Cache
With a dual cache - a local cache and Aerospike CacheDB - in front of another database, the same equation can be applied for the two levels of caching. The hit-ratio and latency-ratio are with respect to each cache's respective data source. 

- Level 1 speedup: 1/(1 - H1 + H1/L1)
- Level 2 speedup: 1/(1 - H2 + H2/L2)
- Combined speedup: 1/(1 - H1 + H1/L1) * 1/(1 - H2 + H2/L2)

For simplicity of illustration and to focus on the impact of a local cache in a dual cache setting, we peg the Aerospike CacheDB values to H2=0.8 and L2=100. As Aerospike provides a large cache size with a sub-millisecond range average access time, these numbers are not atypical. 

The local cache H1 values are varied over (0.1, 0.3, 0.5, 0.7, 0.9) and L1 values over (10, 100, 1000).

In [141]:
def dual_cache_speedup(hit_ratio_1, latency_ratio_1, hit_ratio_2, latency_ratio_2):
    return 1/(1 - hit_ratio_1 + hit_ratio_1 / latency_ratio_1) * 1/(1 - hit_ratio_2 + hit_ratio_2 / latency_ratio_2)

h2 = 0.8
l2 = 100
print('LR2=%3d'%l2, 'HR2=%3.1f'%h2, 'Speedup with Aerospike Cache Alone: %3.0f%%'
      %((single_cache_speedup(h2, l2)-1)*100), sep='\t')

hr1 = [0.1, 0.3, 0.5, 0.7, 0.9]
lr1 = [10, 100, 1000]
print('LR1', 'HR1', 'Dual Cache Speedup', sep='\t')
for l1 in lr1:
    for h1 in hr1:
        print(l1, h1, '%5.0f%%'%((dual_cache_speedup(h1, l1, h2, l2)-1)*100), 
              math.ceil((dual_cache_speedup(h1, l1, h2, l2)-1)*1)*'*', sep="\t")

LR2=100	HR2=0.8	Speedup with Aerospike Cache Alone: 381%
LR1	HR1	Dual Cache Speedup
10	0.1	  428%	*****
10	0.3	  559%	******
10	0.5	  774%	********
10	0.7	 1199%	************
10	0.9	 2430%	*************************
100	0.1	  434%	*****
100	0.3	  584%	******
100	0.5	  852%	*********
100	0.7	 1466%	***************
100	0.9	 4311%	********************************************
1000	0.1	  434%	*****
1000	0.3	  587%	******
1000	0.5	  861%	*********
1000	0.7	 1499%	***************
1000	0.9	 4665%	***********************************************


As can be seen from the numbers above (and should be intuitively clear) that a local cache in a dual cache setting can be very effective especially with large hit ratios. 

# Takeaways
The key takeaways are:
- A local cache can augment the performance especially when the access pattern provide a higher hit ratio (> 0.5). A local cache can be expensive as it has to be replicated at each client host, but can be effective if high hit ratio is achievable with repeated access to small amounts of data.
- Aerospike  provides significant performance and throughput boost over the backend database due to its large cache size (which typically transalates into a higher hit ratio), and a sub-millisecond access time that can be 2 orders of magnitude faster than the backend database. 


# Clean up
Remove data and close the server connection.

In [142]:
client.truncate(namespace, tutorial_set, 0)
# Close the connection to the Aerospike cluster
client.close()
print('Removed tutorial data. Connection closed.')

Removed tutorial data. Connection closed.


# Further Exploration and Resources
Explore further how Aerospike can be deployed as a cache.
- [Blog posts]
- [Other resources]

## Next steps

Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) to run additional Aerospike notebooks. To run a different notebook, download the notebook from the repo to your local machine, and then click on File->Open, and select Upload.